# Data Day 2019 Power Session

Title: Interactive mapping of social vulnerability caused by climate change using R
Authors: Richard Johansen & Mark Chalmers
University of Cincinnati Libraries
4/1/2019

 Social Vulnerability Data: http://artsandsciences.sc.edu/geog/hvri
 Code: https://github.com/RAJohansen/DataDay2019

## Part I: Introduction to R

In [ ]:
# Step 1: R as a Calculator
1 + 3

In [ ]:
### Step 2: Creating objects in R
# Hint alt - is a shortcut for the < - 
x <- 1+2
x
y <- x +1
y

In [ ]:
### Step 3: Getting Help in R
help(mean)

In [ ]:
### Step 4: Viewing & Examinging a Data set
# Lets explore data using a data set thats contained in R
mtcars  <- mtcars

In [ ]:
# View our table
# Or click the df object under the data window
View(mtcars)

In [ ]:
# Use the names() function to return a list the variables 
names(mtcars)

In [ ]:
#Look at the data types or structure of the data
str(mtcars)
# This is very useful when we analyzing or visualizing data
# Make sure your variables are in the appropiate format!!

In [ ]:
## Quick and simple statistical summary of the data
summary(mtcars)

In [ ]:
# Finding values from out data table
# Lets look at column 2 
mtcars[,2]

In [ ]:
# Lets look at row 5
mtcars[5,]

In [ ]:
# What value is in row 5 column 3?
mtcars[5,3]

In [ ]:
# What if we want to know the max mpg
max(mtcars$mpg)

## Part II: Plotting Using Base R

In [ ]:
# Default Plot
plot(mtcars$mpg)

In [ ]:
## Dotchart ##
dotchart(mtcars$mpg, labels=row.names(mtcars))

In [ ]:
## Histogram ##
hist(mtcars$mpg)

In [ ]:
# Colored Histogram with Different Number of Bins
hist(mtcars$mpg, breaks=10)

In [ ]:
## Scatterplot ##
plot(mtcars$wt,mtcars$mpg)

In [ ]:
## Box Plots ##
boxplot(mtcars$mpg~mtcars$cyl)

In [ ]:
# Boxplot with labels
boxplot(mpg~cyl,
        data=mtcars,
        main="Car Milage Data", 
        xlab="Number of Cylinders",
        ylab="Miles Per Gallon")

## Part III: Data Acquisition

In [ ]:
### Step 1: Install & load required packages 
# The packages are already installed in the Binder session,but 
# on your own machine you would install them with the following line.
#install.packages(c("tigris","tmap","tidyverse","tablulizer","dplyr","sf","leaflet"))
library(tigris)
library(tmap)
library(tidyverse)
library(tabulizer)
library(dplyr)
library(sf)
library(leaflet)


In [ ]:
### Step 2: Extract a web PDF
# Explore file location to ensure accuracy:
website <- "http://artsandsciences.sc.edu/geog/hvri/sites/sc.edu.geog.hvri/files/attachments/SoVI_10_14_Website.pdf"
browseURL(url = website)

In [ ]:
# Use URL location to extract pdf as a table
# When you're unfamilar with a function you can use the ?
?extract_tables
Sovi_table <- extract_tables(website)

In [ ]:
# Lets view what exactly is extracted through this process
View(Sovi_table)

In [ ]:
### Step 3: Converting the web-based PDF into csv
# Lets use two more functions to convert the extracted table
# into a more usable and analysis friendly format

final <- do.call(rbind, Sovi_table[-length(Sovi_table)])

In [ ]:
# Reformate table headers by dropping the first row
final <- as.data.frame(final[2:nrow(final), ])

In [ ]:
# Lets lable the column names so they can merged with Census data
headers <- c('GEOID', 'State_FIP', 'County_FIP', 'County_Name', 'CNTY_SoVI', 
             'Percentile')

In [ ]:
# **NOTE** GEOID is the ID code for CENSUS data
# This is mandatory for the next section

### Step 4: Save the table as a csv 
# This is helpful for eliminating redundancy and reproducibility
write.csv(final, file='Data/SoVI.csv', row.names=FALSE)

## Part IV: Mapping in R

In [ ]:
### Step 1: Load spatial objects into R from US Census Tigris files
# In this case we want to load counties
# The tigris package is connected to the Census's database so we can pull data directly
# We want to pull the spatial objects counties and save them as an R object 


# NOTE: this might take a couple minutes due to the size of the file
# Question: How many counties are there in the USA?

# Load USA Counties from tigris package (US CENSUS)
#Counties <- counties()

# Convert Large SpatialPolygonsDataFrame to Simple Feature (sf)
#Counties_sf <- st_as_sf(Counties)

#Select only Florida Counties & Save them
#Counties_FL <- subset(Counties_sf,STATEFP == "12")
#st_write(Counties_FL, dsn = 'Data/Counties_FL.gpkg')

In [ ]:
#Read geopackage
Counties_FL <- st_read('Data/Counties_FL.gpkg')

In [ ]:
### Step 2: Merge SoVI csv with our county region spatial object
# Load data from package location if not currently loaded
# We can start directly from the objects in our working environment
#Or we can load the data saved in Part 1: Step 4
df <- read.csv('Data/SoVI.csv')

In [ ]:
# Create subset of SoVI data for just Florida Counties
df_FL <- subset(df,State_FIP == "12")
#Notice the number of rows is exactly the same as Counties_FL

In [ ]:
# Now that we have both of objects loaded we can merged them using a common field
# This is a very common practice is GIS
# Each object must have the exact same set of unique identifiers for each row
# Using the merge fucntion we can combine the spatial object with our data frame
FL_SoVI <- merge(Counties_FL,df_FL, by = "GEOID", all = FALSE)

In [ ]:
### Step 3: Plot using base plot
# We want to plot the spatial object from the values of the first column
# In this case that is the unique ID for each column
plot(FL_SoVI[1])

In [ ]:
### Step 4: Mapping with tmap
# tmap uses the same grammar of graphics as ggplot
# We build on our graphics like layers on a cake

# Plot  data 
tm_shape(FL_SoVI) +
  tm_fill()

In [ ]:
# Now add our county borders         
tm_shape(FL_SoVI) +
  tm_borders() + 
  tm_fill()


In [ ]:
# Lets add our SoVI data to explore trends         
tm_shape(FL_SoVI) +
  tm_borders() + 
  tm_fill(col = "CNTY_SoVI")

In [ ]:
# Manually define lable breaks
breaks = c(-6,-3,0,3,6)
tm_shape(FL_SoVI) +
  tm_borders() + 
  tm_fill(col = "CNTY_SoVI", breaks = breaks)

In [ ]:
# We can explore color palettes
tmaptools::palette_explorer()

In [ ]:
# Lets choose our own color palette and add a continuous scale bar
tm_shape(FL_SoVI) +
  tm_borders() + 
  tm_fill(col = "CNTY_SoVI", style = "cont", palette = "viridis")


In [ ]:

# Add some cartographic elements
tm_shape(FL_SoVI) +
  tm_borders() + 
  tm_fill(col = "CNTY_SoVI", style = "cont", palette = "viridis") +
  tm_layout(title = "Florida SoVI Vulnerability Index by County",
            legend.outside = FALSE,
            frame = TRUE,
            inner.margins = 0.1,
            legend.title.size = 1.5,
            legend.text.size = 1.1) +
  tm_compass(type = "arrow", position = c("right", "top"), size = 2) +
  tm_scale_bar(breaks = c(0, 100, 200),size = 0.8)
  

In [ ]:
### Finally Lets save our plot

#Saving a plot
#jpeg('My_Awesome_Map.jpg', width = 7, height = 7, units = "in", res =300)

#tm_shape(FL_SoVI) +
#  tm_borders() + 
#  tm_fill(col = "CNTY_SoVI", style = "cont", palette = "viridis") +
#  tm_layout(title = "Florida SoVI Vulnerability Index by County",
#            legend.outside = FALSE,
#            frame = TRUE,
#            inner.margins = 0.1,
#            legend.title.size = 1.5,
#            legend.text.size = 1.1) +
#  tm_compass(type = "arrow", position = c("right", "top"), size = 2) +
#  tm_scale_bar(breaks = c(0, 100, 200),size = 0.8)

#dev.off()


## Part V: Interactive Mapping and leaflet

In [ ]:
# Lets examine the leaflet documentation
?leaflet
browseURL(url = "https://rstudio.github.io/leaflet/")

In [ ]:
### Step 1: Convert Static tmap to Interactive Map using leaflet
# Create R object from map 
map <- tm_shape(FL_SoVI) +
  tm_borders() +
  tm_fill(col = "CNTY_SoVI",
          id = "NAME",
          popup.vars = c("NAME","CNTY_SoVI"))


In [ ]:
#Call that object using tmap_leaflet function
tmap_leaflet(map)

In [ ]:
### Step 2: Complex Mapping with leaflet
# Create duplicate maps so we can do a side by side comparison of SoVI and Flood Zone
FL_pop <- read.csv("Data/FL_Population.csv")

In [ ]:
FL_pop$NAMELSAD <- FL_pop$County
# Merge population data into spatial object 
FL_SoVI <- merge(FL_SoVI,FL_pop, by = "NAMELSAD", all = FALSE)

In [ ]:
#Add Flood zone lines
FL_slr_10ft <- st_read("Data/FL_slr_10ft.gpkg")

In [ ]:
facets  <- c("CNTY_SoVI","Population")
map_facets <- tm_shape(FL_SoVI) +
  tm_polygons(facets) +
  tm_borders() +
  tm_fill(col = "CNTY_SoVI",
          id = "NAME",
          popup.vars = c("NAME","CNTY_SoVI")) +
  tm_shape(FL_slr_10ft) +
    tm_polygons(col = "blue", alpha = 0.5) +
  tm_facets(nrow = 1, sync = TRUE)

In [ ]:
tmap_leaflet(map_facets)

In [ ]:
### Step 3: Adding a basemap to a Interactive Map
map_facets_base <- tm_basemap(leaflet::providers$Esri.WorldImagery) + 
  tm_shape(FL_SoVI) +
  tm_polygons(facets) +
  tm_borders() +
  tm_fill(col = "CNTY_SoVI",
          id = "NAME",
          popup.vars = c("NAME","CNTY_SoVI")) +
  tm_shape(FL_slr_10ft) +
  tm_polygons(col = "blue", alpha = 0.5) +
  tm_facets(nrow = 1, sync = TRUE)

In [ ]:
tmap_leaflet(map_facets_base)

In [ ]:
### Step 4: Interactive Map using leaflet only
pal <- colorNumeric(
  palette = "RdYlBu",
  domain = FL_SoVI$CNTY_SoVI
)

In [ ]:
m <- leaflet(FL_SoVI) %>%
  addTiles(group = "Open Street Map") %>% 
  addProviderTiles(leaflet::providers$Esri.WorldImagery, group = "Satellite Imagery") %>% 
  addPolygons(color = "#444444", weight = 1, smoothFactor = 0.5,
              opacity = 1.0, fillOpacity = 0.5,
              fillColor = ~colorQuantile("RdYlBu", CNTY_SoVI)(CNTY_SoVI),
              highlightOptions = highlightOptions(color = "white", weight = 2,
                                                  bringToFront = TRUE), group = "SoVI") %>% 
  addPolygons(data = FL_slr_10ft, fillColor = "blue", fillOpacity = 0.5, group = "Sea Level Rise") %>% 
  addLayersControl(baseGroups = c("Satellite Imagery", "Open Street Map"),
                   overlayGroups = c("Sea Level Rise", "SoVI"),
                   options = layersControlOptions(collapsed = FALSE)) %>% 
  addLegend("bottomleft",
            title = "Vulnerability (SoVI)",
            pal = pal,
            values = ~CNTY_SoVI,
            opacity = 1)

In [ ]:
#Print the Map
m